In [44]:
import pandas as pd
import numpy as np
import os
import glob
#os.chdir('../src')
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from model.neural_network import ImageRecognition
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from PIL import Image
import numpy as np
os.chdir('..')
def load_images_from_folder(folder_path, target_size=(256, 256), label = None):
    """
    Carga imágenes desde una carpeta, las redimensiona y las convierte en arrays.
    
    Args:
    - folder_path (str): Ruta de la carpeta que contiene las imágenes.
    - target_size (tuple): Tamaño al que se redimensionarán las imágenes (ancho, alto).

    Returns:
    - images_array (numpy.ndarray): Array de imágenes.
    - labels (list): Lista de nombres de archivo (opcionalmente puede ser la clase).
    """
    images_list = []  # Cambiamos el nombre a `images_list` para evitar confusión
    labels = []
    
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            with Image.open(file_path) as img:
                # Redimensionar imagen al tamaño deseado
                img_resized = img.resize(target_size)
                # Convertir a array
                img_array = np.array(img_resized)
                # Asegurarse de que tiene 3 canales (RGB)
                if img_array.ndim == 2:  # Imagen en escala de grises
                    img_array = np.stack((img_array,) * 3, axis=-1)
                elif img_array.shape[-1] == 4:  # Imagen RGBA
                    img_array = img_array[..., :3]
                
                images_list.append(img_array)  # Agregar a la lista
                labels.append(label)  # Podrías extraer clases desde el nombre si es necesario
        except Exception as e:
            print(f"Error al cargar la imagen {file_path}: {e}")
    
    # Convertir la lista final a un array de NumPy
    images_array = np.array(images_list)
    return images_array, labels
images_array = np.empty((0, 256, 256, 3))
labels = []
for i in glob.glob('data_exercise/*'):
    images_array_, labels_ = load_images_from_folder(i, target_size=(256, 256), label = i.split('/')[1])
    images_array = np.concatenate((images_array, images_array_), axis=0)
    labels = labels + labels_

In [5]:
from sklearn.model_selection import train_test_split
images_array_esc = images_array / 255.


x_train, x_test, y_train, y_test = train_test_split(images_array_esc, labels, test_size=0.3, random_state=123)
x_training, x_val, y_training, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=123)

y_train_cat = pd.get_dummies(y_training).values * 1
y_test_cat = pd.get_dummies(y_test).values * 1
y_val_cat = pd.get_dummies(y_val).values * 1

# Prepare datasets and dataloaders
batch_size = 32

# Assuming x_train, y_train_cat, x_test, y_test_cat are numpy arrays
x_train_tensor = torch.tensor(x_training, dtype=torch.float32).permute(0, 3, 1, 2)  # Convert to NCHW
y_train_tensor = torch.tensor(y_train_cat.argmax(axis=1), dtype=torch.long)  # Convert one-hot to class indices

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).permute(0, 3, 1, 2)  # Convert to NCHW
y_val_tensor = torch.tensor(y_val_cat.argmax(axis=1), dtype=torch.long)  # Convert one-hot to class indices


x_test_tensor = torch.tensor(x_test, dtype=torch.float32).permute(0, 3, 1, 2)  # Convert to NCHW
y_test_tensor = torch.tensor(y_test_cat.argmax(axis=1), dtype=torch.long)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [45]:
device = torch.device("cpu")
criterion = nn.CrossEntropyLoss()
model = ImageRecognition(num_classes=6)
model.to(device)

model.fit(criterion, train_loader, val_loader, device, num_epochs=10)

Epoch 1/10, Loss: 37.7846, Val Accuracy: 0.1890
Epoch 2/10, Loss: 37.0830, Val Accuracy: 0.1829
Epoch 3/10, Loss: 34.4192, Val Accuracy: 0.3780
Epoch 4/10, Loss: 28.1050, Val Accuracy: 0.4939
Epoch 5/10, Loss: 20.7523, Val Accuracy: 0.6220
Epoch 6/10, Loss: 15.0838, Val Accuracy: 0.6037


KeyboardInterrupt: 

In [39]:

from sklearn.metrics import accuracy_score
accuracy_score(np.array(model.predict(x_test_tensor, batch_size, device)), np.array([i.item() for i in y_test_tensor]))

0.6866096866096866

In [40]:
model.save_model("custom_vgg16.pth")

'Model saved to custom_vgg16.pth'

In [41]:
model.predict_from_load(x_test_tensor, 
                        "custom_vgg16.pth",
                        device)

tensor([1, 1, 4, 5, 5, 0, 2, 4, 0, 3, 0, 3, 1, 2, 4, 5, 4, 2, 0, 4, 5, 1, 0, 1,
        4, 2, 0, 2, 0, 4, 5, 2, 3, 4, 0, 4, 3, 4, 0, 0, 2, 4, 2, 4, 3, 3, 0, 2,
        3, 3, 1, 4, 1, 0, 0, 1, 1, 2, 2, 0, 1, 5, 2, 3, 0, 0, 1, 0, 5, 3, 3, 1,
        5, 3, 3, 3, 1, 0, 2, 1, 3, 4, 4, 4, 3, 1, 4, 2, 5, 1, 1, 3, 4, 3, 0, 2,
        4, 0, 1, 3, 5, 1, 0, 2, 1, 4, 0, 5, 2, 4, 5, 0, 5, 2, 3, 0, 2, 0, 1, 0,
        1, 1, 0, 0, 1, 1, 0, 4, 4, 5, 3, 2, 5, 3, 0, 2, 0, 3, 0, 0, 0, 2, 5, 4,
        1, 3, 4, 3, 4, 4, 3, 1, 0, 3, 2, 0, 1, 4, 0, 2, 3, 4, 3, 2, 3, 3, 0, 5,
        1, 3, 2, 1, 0, 0, 5, 4, 3, 0, 1, 1, 1, 1, 0, 3, 0, 3, 1, 1, 0, 3, 1, 4,
        5, 3, 0, 0, 2, 4, 3, 3, 5, 0, 3, 1, 1, 2, 5, 0, 1, 1, 2, 3, 4, 4, 0, 0,
        0, 3, 4, 3, 2, 1, 0, 1, 2, 4, 5, 5, 1, 3, 3, 4, 0, 5, 3, 4, 3, 3, 3, 0,
        1, 2, 3, 4, 3, 2, 3, 5, 1, 1, 5, 0, 5, 3, 1, 2, 3, 0, 0, 4, 3, 0, 0, 3,
        0, 3, 3, 3, 0, 2, 0, 0, 5, 0, 3, 2, 4, 3, 0, 2, 5, 3, 1, 1, 5, 0, 5, 2,
        0, 5, 0, 4, 1, 3, 3, 3, 2, 2, 3,